# <center><b> Python Programming in Energy Science I</b></center>

## <center> Assignment A2 </center>

### <center>[Group 4]</center>
<center> Juan Manuel Boullosa Novo </center>

-------------------------------

### E1. Data analysis

### E1. Data analysis

In [60]:
def read_csv(filepath):
    with open(filepath, 'r') as file:
        next(file) # skip header
        data = [line.strip().split(',') for line in file]
    datestamp = [row[0] for row in data]
    wind_speed_vectors = [[float(row[1]), float(row[2])] for row in data]
    return datestamp, wind_speed_vectors

def mean_vector(wind_speed_vectors):
    vector_length = len(wind_speed_vectors)
    sum_u100, sum_v100 = sum(float(row[0]) for row in wind_speed_vectors), sum(float(row[1]) for row in wind_speed_vectors)
    avg_vector = [sum_u100/vector_length, sum_v100/vector_length]
    avg_vector_magnitude = vector_magnitude(avg_vector)
    std_dev = vector_standard_deviation(wind_speed_vectors, avg_vector)
    return avg_vector, avg_vector_magnitude, std_dev

def vector_magnitude(vector):
    return (vector[0]**2 + vector[1]**2)**0.5

def vector_standard_deviation(wind_speed_vectors, mean_vector):
    squared_differences = [(float(vector[0])-mean_vector[0])**2 + (float(vector[1])-mean_vector[1])**2 for vector in wind_speed_vectors]
    variance = sum(squared_differences)/len(squared_differences)
    return variance**0.5

def check_wind_speeds(wind_speed_vectors):
    magnitudes = [vector_magnitude(vector) for vector in wind_speed_vectors]
    return all(mag > 0 for mag in magnitudes), any(mag > 25 for mag in magnitudes)

def find_min_max(wind_speed_vectors):
    magnitudes = [vector_magnitude(vector) for vector in wind_speed_vectors]
    return min(magnitudes), max(magnitudes)

def bin_counter(wind_speed_vectors, min_wind_speed, max_wind_speed, bin_size=1):
    bins = dict()
    for bin in float_range(min_wind_speed, max_wind_speed, bin_size):
        bins[bin] = sum(1 for vector in wind_speed_vectors if bin <= vector_magnitude(vector) < bin+1)
    return bins

def float_range(start, stop, step):
    # Custom float range function
    frange = []
    while start < stop:
        frange.append(start)
        start += step
    return frange

def longest_period_greater_than(datestamp, wind_speed_vectors, threshold):
    longest_period, current_period = 0, 0
    previous_date = None
    
    for i in range(len(wind_speed_vectors)):
        magnitude = vector_magnitude(wind_speed_vectors[i])
        date = datestamp[i]
        
        if magnitude > threshold:
            if previous_date is not None:
                current_period += time_difference(previous_date, date)
            previous_date = date
        else:
            longest_period = max(longest_period, current_period)
            current_period = 0
    
    longest_period = max(longest_period, current_period)
    return longest_period

def time_difference(date1, date2):
    # Calculate the difference in seconds between two dates
    year1, month1, day1, hour1, minute1, second1 = map(int, date1.replace('-', ' ').replace(':', ' ').split())
    year2, month2, day2, hour2, minute2, second2 = map(int, date2.replace('-', ' ').replace(':', ' ').split())

    time1 = second1 + minute1*60 + hour1*3600 + day1*86400 + month1*2592000 + year1*31104000
    time2 = second2 + minute2*60 + hour2*3600 + day2*86400 + month2*2592000 + year2*31104000

    return abs(time2 - time1)

def s_to_hms(seconds):
    # Convert seconds to hours, minutes, seconds
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return hours, minutes, seconds

datestamp, wind_speed_vectors = read_csv('winddata.csv')
avg_wind_speed_vector, avg_wind_speed_magnitude, wind_speed_standard_deviation = mean_vector(wind_speed_vectors)
all_greater_than_zero, any_greater_than_25 = check_wind_speeds(wind_speed_vectors)
min_wind_speed, max_wind_speed = find_min_max(wind_speed_vectors)
bins = bin_counter(wind_speed_vectors, min_wind_speed, max_wind_speed)
longest_greater_than_10 = longest_period_greater_than(datestamp, wind_speed_vectors, 10)
longest_greater_than_15 = longest_period_greater_than(datestamp, wind_speed_vectors, 15)
longest_greater_than_25 = longest_period_greater_than(datestamp, wind_speed_vectors, 25)

print(*s_to_hms(longest_greater_than_10))



737 0 0


### E2. 

### E3. 

### E4. 

### E5. 

### E6. 